# 1. Initialize the environment

In [ ]:
!pip install --upgrade pip
!pip install langchain openai

In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType

Downloading SQLite3 version of Microsoft's Northwind Database from [jpwhite3 repo](https://github.com/jpwhite3/northwind-SQLite3)

In [ ]:
!curl -OL https://raw.githubusercontent.com/jpwhite3/northwind-SQLite3/master/dist/northwind.db

In [2]:
OPEN_AI_KEY = "YourOpenAIAPIKey"

In [16]:
db = SQLDatabase.from_uri("sqlite:///northwind.db")
toolkit = SQLDatabaseToolkit(
    db=db,
    llm=OpenAI(
        temperature=0,
        openai_api_key=OPEN_AI_KEY,
    ),
)

agent_executor = create_sql_agent(
    llm=OpenAI(
        temperature=0,
        openai_api_key=OPEN_AI_KEY,
    ),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    openai_api_key=OPEN_AI_KEY,
)

# 2. Agent usage

## 2.1. The country with the most expensive costs 

In [17]:
agent_executor.run(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Orders and Customers tables.
Action: sql_db_schema
Action Input: Orders, Customers
Observation: 
CREATE TABLE "Customers" (
	"CustomerID" TEXT, 
	"CompanyName" TEXT, 
	"ContactName" TEXT, 
	"ContactTitle" TEXT, 
	"Address" TEXT, 
	"City" TEXT, 
	"Region" TEXT, 
	"PostalCode" TEXT, 
	"Country" TEXT, 
	"Phone" TEXT, 
	"Fax" TEXT, 
	PRIMARY KEY ("CustomerID")
)

/*
3 rows from Customers table:
CustomerID	CompanyName	ContactName	ContactTitle	Address	City	Region	PostalCode	Country	Phone	Fax
ALFKI	Alfreds Futterkiste	Maria Anders	Sales Representative	Obere Str. 57	Berlin	Western Europe	12209	Germany	030-0074321	030-0076545
ANATR	Ana Trujillo Emparedados y helados	Ana Trujillo	Owner	Avda

'The customers from USA spent the most, with a total of $13,771.29.'

As we can see, LangChain list all tables, columns, and relations between them. The LangChain's comments are pretty clear to understand what each query does.


## 2.2. Average price of the products

In [19]:
agent_executor.run(
    "What is the average price of products?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Products table.
Action: sql_db_schema
Action Input: Products
Observation: 
CREATE TABLE "Products" (
	"ProductID" INTEGER NOT NULL, 
	"ProductName" TEXT NOT NULL, 
	"SupplierID" INTEGER, 
	"CategoryID" INTEGER, 
	"QuantityPerUnit" TEXT, 
	"UnitPrice" NUMERIC DEFAULT 0, 
	"UnitsInStock" INTEGER DEFAULT 0, 
	"UnitsOnOrder" INTEGER DEFAULT 0, 
	"ReorderLevel" INTEGER DEFAULT 0, 
	"Discontinued" TEXT DEFAULT '0' NOT NULL, 
	PRIMARY KEY ("ProductID"), 
	FOREIGN KEY("SupplierID") REFERENCES "Suppliers" ("SupplierID"), 
	FOREIGN KEY("CategoryID") REFERENCES "Categories" ("CategoryID"), 
	CHECK ([UnitPrice]>=(0)), 
	CHECK ([ReorderLevel]>=(0)), 
	CHECK ([UnitsInStock]>=(0)), 
	CHECK ([Uni

'The average price of products is 28.87.'

LangChain understands the context, make the same research as at the previous step, and provide the correct answer.

## 2.3. Total clients number

In [20]:
agent_executor.run(
    "How many customers are there in the Northwind database?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: The Customers table looks like it might have the information I need.
Action: sql_db_schema
Action Input: Customers
Observation: 
CREATE TABLE "Customers" (
	"CustomerID" TEXT, 
	"CompanyName" TEXT, 
	"ContactName" TEXT, 
	"ContactTitle" TEXT, 
	"Address" TEXT, 
	"City" TEXT, 
	"Region" TEXT, 
	"PostalCode" TEXT, 
	"Country" TEXT, 
	"Phone" TEXT, 
	"Fax" TEXT, 
	PRIMARY KEY ("CustomerID")
)

/*
3 rows from Customers table:
CustomerID	CompanyName	ContactName	ContactTitle	Address	City	Region	PostalCode	Country	Phone	Fax
ALFKI	Alfreds Futterkiste	Maria Anders	Sales Representative	Obere Str. 57	Berlin	Western Europe	12209	Germany	030-0074321	030-0076545
ANATR	Ana Trujillo Emparedados y helados	Ana Trujillo	Owner	Avda.

'There are 93 customers in the Northwind database.'

## 2.4. The most expensive product

In [21]:
agent_executor.run(
    "What is the most expensive product in the Northwind database?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Products table.
Action: sql_db_schema
Action Input: Products
Observation: 
CREATE TABLE "Products" (
	"ProductID" INTEGER NOT NULL, 
	"ProductName" TEXT NOT NULL, 
	"SupplierID" INTEGER, 
	"CategoryID" INTEGER, 
	"QuantityPerUnit" TEXT, 
	"UnitPrice" NUMERIC DEFAULT 0, 
	"UnitsInStock" INTEGER DEFAULT 0, 
	"UnitsOnOrder" INTEGER DEFAULT 0, 
	"ReorderLevel" INTEGER DEFAULT 0, 
	"Discontinued" TEXT DEFAULT '0' NOT NULL, 
	PRIMARY KEY ("ProductID"), 
	FOREIGN KEY("SupplierID") REFERENCES "Suppliers" ("SupplierID"), 
	FOREIGN KEY("CategoryID") REFERENCES "Categories" ("CategoryID"), 
	CHECK ([UnitPrice]>=(0)), 
	CHECK ([ReorderLevel]>=(0)), 
	CHECK ([UnitsInStock]>=(0)), 
	CHECK ([Uni

'The most expensive product in the Northwind database is Côte de Blaye, with a UnitPrice of 263.5.'

Okay, that was quite easy, because it requires only one table and one query.
Let's try to ask more complex questions.

We have a `OrderDetails` table, where was information about each order (including discounts). This table has a relation with `Orders` table, where we can find information about customers. And `Customers` table has information about countries.

Let's try to find the country with the biggest discount in total.

## 2.5. Country with the biggest discount

In [22]:
agent_executor.run(
    "Which country got the most discounts?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the Orders table to get the discounts
Action: sql_db_schema
Action Input: Orders
Observation: 
CREATE TABLE "Orders" (
	"OrderID" INTEGER NOT NULL, 
	"CustomerID" TEXT, 
	"EmployeeID" INTEGER, 
	"OrderDate" DATETIME, 
	"RequiredDate" DATETIME, 
	"ShippedDate" DATETIME, 
	"ShipVia" INTEGER, 
	"Freight" NUMERIC DEFAULT 0, 
	"ShipName" TEXT, 
	"ShipAddress" TEXT, 
	"ShipCity" TEXT, 
	"ShipRegion" TEXT, 
	"ShipPostalCode" TEXT, 
	"ShipCountry" TEXT, 
	PRIMARY KEY ("OrderID"), 
	FOREIGN KEY("ShipVia") REFERENCES "Shippers" ("ShipperID"), 
	FOREIGN KEY("CustomerID") REFERENCES "Customers" ("CustomerID"), 
	FOREIGN KEY("EmployeeID") REFERENCES "Employees" ("EmployeeID")
)

/*
3 rows from Orders table:
Ord

'USA got the most discounts with 122.'

It use wrong table, and as a result we got wrong answer. Let's try to fix it. We will rephrase the question: most -> biggest

In [23]:
agent_executor.run(
    "Which country got the biggest discount?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the Orders table to get the discount information.
Action: sql_db_schema
Action Input: Orders
Observation: 
CREATE TABLE "Orders" (
	"OrderID" INTEGER NOT NULL, 
	"CustomerID" TEXT, 
	"EmployeeID" INTEGER, 
	"OrderDate" DATETIME, 
	"RequiredDate" DATETIME, 
	"ShippedDate" DATETIME, 
	"ShipVia" INTEGER, 
	"Freight" NUMERIC DEFAULT 0, 
	"ShipName" TEXT, 
	"ShipAddress" TEXT, 
	"ShipCity" TEXT, 
	"ShipRegion" TEXT, 
	"ShipPostalCode" TEXT, 
	"ShipCountry" TEXT, 
	PRIMARY KEY ("OrderID"), 
	FOREIGN KEY("ShipVia") REFERENCES "Shippers" ("ShipperID"), 
	FOREIGN KEY("CustomerID") REFERENCES "Customers" ("CustomerID"), 
	FOREIGN KEY("EmployeeID") REFERENCES "Employees" ("EmployeeID")
)

/*
3 rows from Order

'Germany got the biggest discount with 1007.64.'

Still wrong :(

Let's try to add more context about tables.

In [8]:
agent_executor.run(
    "Which country got the biggest discount?"
    "Discount information is in the Order Details table."
    "Do not forgive to use quotemarks for table names."
    "Information about countries is in the Customers table."
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Order Details and Customers tables.
Action: sql_db_schema
Action Input: 'Order Details, Customers'
Observation: Error: table_names {"Customers'", "'Order Details"} not found in database
Thought: I should check the table names I'm using.
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Order_Details and Customers tables.
Action: sql_db_schema
Action Input: 'Order_Details, Customers'
Observation: Error: table_names {"Customers'", "'Order_Details"} not found in data

'Agent stopped due to iteration limit or time limit.'

Haha, it falls into the forever loop, because it doesn't know how to handle table name with spaces.

Let's rename `Order Details` table to `OrderDetails` and try again.

In [ ]:
db.run("DROP VIEW IF EXISTS 'ProductDetails_V';")

db.run("ALTER TABLE 'Order Details' RENAME TO OrderDetails;")

In [17]:
agent_executor.run(
    'Which country got the biggest discount?'
    'Discount information is in the OrderDetails table.'
    'Information about countries is in the Customers table.'
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, OrderDetails, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the OrderDetails and Customers tables.
Action: sql_db_schema
Action Input: OrderDetails, Customers
Observation: 
CREATE TABLE "Customers" (
	"CustomerID" TEXT, 
	"CompanyName" TEXT, 
	"ContactName" TEXT, 
	"ContactTitle" TEXT, 
	"Address" TEXT, 
	"City" TEXT, 
	"Region" TEXT, 
	"PostalCode" TEXT, 
	"Country" TEXT, 
	"Phone" TEXT, 
	"Fax" TEXT, 
	PRIMARY KEY ("CustomerID")
)

/*
3 rows from Customers table:
CustomerID	CompanyName	ContactName	ContactTitle	Address	City	Region	PostalCode	Country	Phone	Fax
ALFKI	Alfreds Futterkiste	Maria Anders	Sales Representative	Obere Str. 57	Berlin	Western Europe	12209	Germany	030-0074321	030-0076545
ANATR	Ana Trujillo Emparedados y helados	Ana Trujillo

"I don't know"

> I don't know

Good to hear -_-

We will add a hint to find all relations between tables, and try to find out that
 `OrderDetails.OrderID` is related to `Orders.OrderID`, and `Orders.CustomerID` is related to `Customers.CustomerID`, and `Customers.Country` is what we need.

In [18]:
agent_executor.run(
    'View all tables and their columns.'
    'Find relations between them.'
    'Which country got the biggest discount?'
    'Discount information is in the OrderDetails table.'
    'Information about countries is in the Customers table.'
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, OrderDetails, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the OrderDetails and Customers tables.
Action: sql_db_schema
Action Input: OrderDetails, Customers
Observation: 
CREATE TABLE "Customers" (
	"CustomerID" TEXT, 
	"CompanyName" TEXT, 
	"ContactName" TEXT, 
	"ContactTitle" TEXT, 
	"Address" TEXT, 
	"City" TEXT, 
	"Region" TEXT, 
	"PostalCode" TEXT, 
	"Country" TEXT, 
	"Phone" TEXT, 
	"Fax" TEXT, 
	PRIMARY KEY ("CustomerID")
)

/*
3 rows from Customers table:
CustomerID	CompanyName	ContactName	ContactTitle	Address	City	Region	PostalCode	Country	Phone	Fax
ALFKI	Alfreds Futterkiste	Maria Anders	Sales Representative	Obere Str. 57	Berlin	Western Europe	12209	Germany	030-0074321	030-0076545
ANATR	Ana Trujillo Emparedados y helados	Ana Trujillo

"I don't know"

Maybe we give too much information, let's try to remove some of them.
Hope LangChain is smart enough.

In [19]:
agent_executor.run(
    'View all tables and their columns.'
    'Find relations between them.'
    'Which country got the biggest discount?'
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, OrderDetails, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the relevant tables
Action: sql_db_schema
Action Input: "Orders, OrderDetails"
Observation: 
CREATE TABLE "OrderDetails" (
	"OrderID" INTEGER NOT NULL, 
	"ProductID" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC DEFAULT 0 NOT NULL, 
	"Quantity" INTEGER DEFAULT 1 NOT NULL, 
	"Discount" REAL DEFAULT 0 NOT NULL, 
	PRIMARY KEY ("OrderID", "ProductID"), 
	FOREIGN KEY("ProductID") REFERENCES "Products" ("ProductID"), 
	FOREIGN KEY("OrderID") REFERENCES "Orders" ("OrderID"), 
	CHECK ([Discount]>=(0) AND [Discount]<=(1)), 
	CHECK ([Quantity]>(0)), 
	CHECK ([UnitPrice]>=(0))
)

/*
3 rows from OrderDetails table:
OrderID	ProductID	UnitPrice	Quantity	Discount
10248	11	14.0000000000	12	0.0
10248	42	9.

'Germany got the biggest discount.'

We get an answer. Let's check it manually.

```SQL
SELECT Customers.Country, SUM(OrderDetails.Discount) AS TotalDiscount 
FROM OrderDetails
INNER JOIN Orders ON OrderDetails.OrderID = Orders.OrderID 
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID 
GROUP BY Customers.Country 
ORDER BY TotalDiscount DESC 
LIMIT 10;
```

| Country   | TotalDiscount      |
| --------- | ------------------ |
| USA       | 20.939999999999998 |
| Germany   | 20.700000000000017 |
| Brazil    | 13.500000000000004 |
| France    | 10.149999999999999 |
| Austria   | 8.600000000000001  |
| Venezuela | 7.900000000000001  |
| Sweden    | 6.8500000000000005 |
| Ireland   | 6.25               |
| Canada    | 4.799999999999999  |
| UK        | 3.1                |


We get the wrong answer, because it counts the number of orders with discounts, not the total sum of them. 

Potential problem is with our prompt. Let's try to rephrase the question.

In [20]:
agent_executor.run(
    'View all tables and their columns.'
    'Find relations between them.'
    'Which country got the biggest discount in total?'
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, OrderDetails, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Orders and OrderDetails tables.
Action: sql_db_schema
Action Input: "Orders, OrderDetails"
Observation: 
CREATE TABLE "OrderDetails" (
	"OrderID" INTEGER NOT NULL, 
	"ProductID" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC DEFAULT 0 NOT NULL, 
	"Quantity" INTEGER DEFAULT 1 NOT NULL, 
	"Discount" REAL DEFAULT 0 NOT NULL, 
	PRIMARY KEY ("OrderID", "ProductID"), 
	FOREIGN KEY("ProductID") REFERENCES "Products" ("ProductID"), 
	FOREIGN KEY("OrderID") REFERENCES "Orders" ("OrderID"), 
	CHECK ([Discount]>=(0) AND [Discount]<=(1)), 
	CHECK ([Quantity]>(0)), 
	CHECK ([UnitPrice]>=(0))
)

/*
3 rows from OrderDetails table:
OrderID	ProductID	UnitPrice	Quantity	Discount
10248	11	14.0000000000	12

'USA got the biggest discount in total.'

Yaaaaaay, it works!

## 2.6. Top 3 countries by the sum of orders

In [23]:
agent_executor.run(
    'View all tables and their columns.'
    'Find relations between them.'
    'Select top 3 countries by the sum of orders?'
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, OrderDetails, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Orders and OrderDetails tables.
Action: sql_db_schema
Action Input: "Orders, OrderDetails"
Observation: 
CREATE TABLE "OrderDetails" (
	"OrderID" INTEGER NOT NULL, 
	"ProductID" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC DEFAULT 0 NOT NULL, 
	"Quantity" INTEGER DEFAULT 1 NOT NULL, 
	"Discount" REAL DEFAULT 0 NOT NULL, 
	PRIMARY KEY ("OrderID", "ProductID"), 
	FOREIGN KEY("ProductID") REFERENCES "Products" ("ProductID"), 
	FOREIGN KEY("OrderID") REFERENCES "Orders" ("OrderID"), 
	CHECK ([Discount]>=(0) AND [Discount]<=(1)), 
	CHECK ([Quantity]>(0)), 
	CHECK ([UnitPrice]>=(0))
)

/*
3 rows from OrderDetails table:
OrderID	ProductID	UnitPrice	Quantity	Discount
10248	11	14.0000000000	12

'The top 3 countries by the sum of orders are USA, Germany, and Austria.'

## 2.7. Top 3 countries by the number of orders

In [24]:
agent_executor.run(
    'View all tables and their columns.'
    'Find relations between them.'
    'Select top 3 countries by the number of orders?'
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, OrderDetails, Orders, Products, Regions, Shippers, Suppliers, Territories
Thought: I should query the schema of the Orders and OrderDetails tables.
Action: sql_db_schema
Action Input: "Orders, OrderDetails"
Observation: 
CREATE TABLE "OrderDetails" (
	"OrderID" INTEGER NOT NULL, 
	"ProductID" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC DEFAULT 0 NOT NULL, 
	"Quantity" INTEGER DEFAULT 1 NOT NULL, 
	"Discount" REAL DEFAULT 0 NOT NULL, 
	PRIMARY KEY ("OrderID", "ProductID"), 
	FOREIGN KEY("ProductID") REFERENCES "Products" ("ProductID"), 
	FOREIGN KEY("OrderID") REFERENCES "Orders" ("OrderID"), 
	CHECK ([Discount]>=(0) AND [Discount]<=(1)), 
	CHECK ([Quantity]>(0)), 
	CHECK ([UnitPrice]>=(0))
)

/*
3 rows from OrderDetails table:
OrderID	ProductID	UnitPrice	Quantity	Discount
10248	11	14.0000000000	12

'The top 3 countries by the number of orders are USA, Germany, and Brazil.'